In [31]:
import pandas as pd
import numpy as np
from dataProcessingUtils import clean_phrase

In [32]:
# new labels
path = "data/to_label/"
test = pd.read_csv(path + "test_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
test["label"] = pd.to_numeric(test["label"], errors='coerce')
train_1 = pd.read_csv(path + "train_first_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
train_2 = pd.read_csv(path + "train_second_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]
extra_neg = pd.read_csv(path + "extra_neg.csv")[["Unnamed: 0",'restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect1','new_aspect2']]
extra_neg = extra_neg.rename(columns= {'new_aspect1':'new_aspect_1', 'new_aspect2':'new_aspect_2'})
val = pd.read_csv(path + "val_subset.csv")[['restaurant_code','review_title','review_body','account_name','account_id','aspect','phrase','label','new_aspect_1','new_aspect_2']]

index = extra_neg.loc[extra_neg["Unnamed: 0"] == 0].index

test = pd.concat([test,extra_neg.iloc[:index[1],1:]],axis=0)
train = pd.concat([train_1,train_2,extra_neg.iloc[index[1] + 1:index[2],1:]],axis=0)
val = pd.concat([val,extra_neg.iloc[index[2] + 1:,1:]],axis=0)
train.shape,val.shape,test.shape

((8348, 10), (3055, 10), (3040, 10))

In [33]:
for data in [train_1,train_2,train,val,test]:
    print("Total size = {}".format(data.shape[0]))
    print("Noise = {} ".format(data.loc[(data.label == 0.0) & (data.new_aspect_1.isnull()) & (data.new_aspect_2.isnull())].shape[0]))
    print("Aspect with no adjectives = {} ".format(data.loc[(data.new_aspect_1.notnull()) & (data.new_aspect_2.isnull())].shape[0]))
    print("Aspect with adjectives = {} ".format(data.loc[(data.new_aspect_1.notnull()) & (data.new_aspect_2.notnull())].shape[0]))
    print("")

Total size = 3108
Noise = 156 
Aspect with no adjectives = 329 
Aspect with adjectives = 265 

Total size = 3108
Noise = 894 
Aspect with no adjectives = 1052 
Aspect with adjectives = 1157 

Total size = 8348
Noise = 1670 
Aspect with no adjectives = 2110 
Aspect with adjectives = 2204 

Total size = 3055
Noise = 650 
Aspect with no adjectives = 892 
Aspect with adjectives = 1512 

Total size = 3040
Noise = 707 
Aspect with no adjectives = 1005 
Aspect with adjectives = 1304 



In [34]:
test.head()

,restaurant_code,review_title,review_body,account_name,account_id,aspect,phrase,label,new_aspect_1,new_aspect_2
0,328-katong-laksa-united-square,328 Katong Laksa; Now you don't have to travel...,Back then during the 2013 hawker heroes challe...,Qing Xiang,@qingxiang,food,"katong, laksa, travel, way",0.0,NaN,NaN
1,328-katong-laksa-united-square,328 Katong Laksa; Now you don't have to travel...,Back then during the 2013 hawker heroes challe...,Qing Xiang,@qingxiang,food,"east, centralised, outlet, united, square, sho...",0.0,NaN,NaN
2,328-katong-laksa-united-square,328 Katong Laksa; Now you don't have to travel...,Back then during the 2013 hawker heroes challe...,Qing Xiang,@qingxiang,food,"laksa, defeated, gordon, ramsay, version",1.0,food,food
3,328-katong-laksa-united-square,328 Katong Laksa; Now you don't have to travel...,Back then during the 2013 hawker heroes challe...,Qing Xiang,@qingxiang,food,"laksa, category",0.0,NaN,NaN
4,328-katong-laksa-united-square,328 Katong Laksa; Now you don't have to travel...,Back then during the 2013 hawker heroes challe...,Qing Xiang,@qingxiang,food,"definitely, live",1.0,food,food


####  Data Preprocessing

1. Clean Phrase (Utils) 
2. Filter the phrase column where there is an empty string 
3. Combine the aspects into 1 & aggregate the sentiment scores by averaging then sign(>0 = 1,=0 =0 <0 = -1)


In [35]:
# clean phrases
train["phrase_lemma"] = train.phrase.apply(lambda x: clean_phrase(x, lemmatize=True, stem=False))
val["phrase_lemma"] = val.phrase.apply(lambda x: clean_phrase(x, lemmatize=True, stem=False))
test["phrase_lemma"] = test.phrase.apply(lambda x: clean_phrase(x, lemmatize=True, stem=False))

train["phrase_stem"] = train.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=True))
val["phrase_stem"] = val.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=True))
test["phrase_stem"] = test.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=True))

train["phrase_emoticon_generic"] = train.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=False, convert_emoji_generic = True))
val["phrase_emoticon_generic"] = val.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=False, convert_emoji_generic = True))
test["phrase_emoticon_generic"] = test.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=False, convert_emoji_generic = True))

train["phrase_lemma_emoticon_generic"] = train.phrase.apply(lambda x: clean_phrase(x, lemmatize=True, stem=False, convert_emoji_generic = True))
val["phrase_lemma_emoticon_generic"] = val.phrase.apply(lambda x: clean_phrase(x, lemmatize=True, stem=False, convert_emoji_generic = True))
test["phrase_lemma_emoticon_generic"] = test.phrase.apply(lambda x: clean_phrase(x, lemmatize=True, stem=False, convert_emoji_generic = True))

train["phrase_stem_emoticon_generic"] = train.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=True, convert_emoji_generic = True))
val["phrase_stem_emoticon_generic"] = val.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=True, convert_emoji_generic = True))
test["phrase_stem_emoticon_generic"] = test.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=True, convert_emoji_generic = True))

train["phrase_emoticon_unique"] = train.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=False, convert_emoji_unique = True))
val["phrase_emoticon_unique"] = val.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=False, convert_emoji_unique = True))
test["phrase_emoticon_unique"] = test.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=False, convert_emoji_unique = True))

train["phrase_lemma_emoticon_unique"] = train.phrase.apply(lambda x: clean_phrase(x, lemmatize=True, stem=False, convert_emoji_unique = True))
val["phrase_lemma_emoticon_unique"] = val.phrase.apply(lambda x: clean_phrase(x, lemmatize=True, stem=False, convert_emoji_unique = True))
test["phrase_lemma_emoticon_unique"] = test.phrase.apply(lambda x: clean_phrase(x, lemmatize=True, stem=False, convert_emoji_unique = True))

train["phrase_stem_emoticon_unique"] = train.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=True, convert_emoji_unique = True))
val["phrase_stem_emoticon_unique"] = val.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=True, convert_emoji_unique = True))
test["phrase_stem_emoticon_unique"] = test.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=True, convert_emoji_unique = True))

train.phrase = train.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=False))
val.phrase = val.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=False))
test.phrase = test.phrase.apply(lambda x: clean_phrase(x, lemmatize=False, stem=False))

In [36]:
# filter out rows where there are no characters
train = train.loc[(train.phrase.str.len() > 0)]
val = val.loc[(val.phrase.str.len() > 0)]
test = test.loc[(test.phrase.str.len() > 0)]

In [37]:
# if nan label, replace with 0
train.label = train.label.apply(lambda x: 0 if np.isnan(x) else x)
val.label = val.label.apply(lambda x: 0 if np.isnan(x) else x)
test.label = test.label.apply(lambda x: 0 if np.isnan(x) else x)

/Users/xinminaw/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [38]:
trainagg = train.groupby(["restaurant_code","review_title","account_name","new_aspect_1"]).agg({"phrase": " ".join,
                                                                                                "phrase_lemma":" ".join,
                                                                                                "phrase_stem":" ".join,
                                                                                                "phrase_emoticon_generic" : " ".join,
                                                                                                "phrase_emoticon_unique" : " ".join,
                                                                                                "phrase_stem_emoticon_generic" : " ".join,
                                                                                                "phrase_lemma_emoticon_generic": " ".join,
                                                                                                "phrase_stem_emoticon_unique": " ".join,
                                                                                                "phrase_lemma_emoticon_unique" : " ".join,                                                                                
                                                                                                "label": np.mean})                                                                                                
trainagg = trainagg.reset_index()
trainagg.label = trainagg.label.map(np.sign)

In [39]:
valagg = val.groupby(["restaurant_code","review_title","account_name","new_aspect_1"]).agg({"phrase": " ".join,
                                                                                               "phrase_lemma":" ".join,
                                                                                               "phrase_stem":" ".join,
                                                                                               "phrase_emoticon_generic" : " ".join,
                                                                                                "phrase_emoticon_unique" : " ".join,
                                                                                                "phrase_stem_emoticon_generic" : " ".join,
                                                                                                "phrase_lemma_emoticon_generic": " ".join,
                                                                                                "phrase_stem_emoticon_unique": " ".join,
                                                                                                "phrase_lemma_emoticon_unique" : " ".join,                                                                                  
                                                                                               "label": np.mean})   
valagg = valagg.reset_index()
valagg.label = valagg.label.map(np.sign)

In [40]:
testagg = test.groupby(["restaurant_code","review_title","account_name","new_aspect_1"]).agg({"phrase": " ".join,
                                                                                               "phrase_lemma":" ".join,
                                                                                               "phrase_stem":" ".join,
                                                                                               "phrase_emoticon_generic" : " ".join,
                                                                                                "phrase_emoticon_unique" : " ".join,
                                                                                                "phrase_stem_emoticon_generic" : " ".join,
                                                                                                "phrase_lemma_emoticon_generic": " ".join,
                                                                                                "phrase_stem_emoticon_unique": " ".join,
                                                                                                "phrase_lemma_emoticon_unique" : " ".join,                                                                                 
                                                                                               "label": np.mean})   
testagg = testagg.reset_index()
testagg.label = testagg.label.map(np.sign)
testagg.label.value_counts()

 1.0    453
 0.0    218
-1.0    104
Name: label, dtype: int64

In [41]:
trainvalagg = pd.concat([trainagg, valagg],axis=0)

In [ ]:
path = "data/labelled_data/"
trainagg.to_csv(path + "train_newpreproc_emoticon.csv")
valagg.to_csv(path + "val_newpreproc_emoticon.csv")
testagg.to_csv(path + "test_newpreproc_emoticon.csv")

##### Text Vectorisation 
1. BoW
2. Tf-idf


In [42]:
path = "data/labelled_data/"
trainagg = pd.read_csv(path + "train_newpreproc_emoticon.csv")
valagg = pd.read_csv(path + "val_newpreproc_emoticon.csv")
testagg = pd.read_csv(path + "test_newpreproc_emoticon.csv")
trainvalagg = pd.concat([trainagg, valagg],axis=0)

In [43]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
import nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()
import re
from tqdm import tqdm

In [44]:
bow = CountVectorizer(analyzer='word',lowercase=True,ngram_range=(1, 3))
bow_train = bow.fit_transform(trainvalagg.phrase)
bow_test = bow.transform(testagg.phrase)

In [45]:
tfidf = TfidfVectorizer(analyzer='word',lowercase=True,ngram_range=(1, 3))
tfidf_train = tfidf.fit_transform(trainvalagg.phrase)
tfidf_test = tfidf.transform(testagg.phrase)

##### Modelling (baseline)

Naive Bayes/Support Vector Machines (NBSVM)/Logistic Regression

In [46]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.svm import SVC

logreg = LogisticRegression()
nb = MultinomialNB()
svm = SVC()

all_train = [bow_train,tfidf_train]
all_test = [bow_test,tfidf_test]
all_vectorisers = ["BoW","TfIdf"]

for train,test,vectoriser in zip(all_train,all_test,all_vectorisers):
    logreg.fit(train,trainvalagg.label)
    
    print("Logistic Regression + " + vectoriser)
    print("Training Data")
    print(classification_report(trainvalagg.label, logreg.predict(train)))
    print("Test Data")
    print(classification_report(testagg.label, logreg.predict(test),digits=4)) 
    
    svm.fit(train,trainvalagg.label)
    
    print("SVM + " + vectoriser)
    print("Training Data")
    print(classification_report(trainvalagg.label, svm.predict(train)))
    print("Test Data")
    print(classification_report(testagg.label, svm.predict(test),digits=4)) 

    # Fasttest dont work w NaiveBayes
    nb.fit(train,trainvalagg.label)
    print("Naive Bayes + "+ vectoriser)
    print("Training Data")
    print(classification_report(trainvalagg.label, nb.predict(train)))
    print("Test Data")
    print(classification_report(testagg.label, nb.predict(test),digits=4))    

Logistic Regression + BoW
Training Data
              precision    recall  f1-score   support

        -1.0       0.99      0.96      0.98       333
         0.0       0.98      0.99      0.99       917
         1.0       0.99      0.99      0.99      1236

    accuracy                           0.99      2486
   macro avg       0.99      0.98      0.99      2486
weighted avg       0.99      0.99      0.99      2486

Test Data
              precision    recall  f1-score   support

        -1.0     0.7021    0.3173    0.4371       104
         0.0     0.5000    0.7982    0.6148       218
         1.0     0.8368    0.7020    0.7635       453

    accuracy                         0.6774       775
   macro avg     0.6797    0.6058    0.6051       775
weighted avg     0.7240    0.6774    0.6779       775

SVM + BoW
Training Data
              precision    recall  f1-score   support

        -1.0       0.97      0.66      0.78       333
         0.0       0.79      0.97      0.87       917
 